In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Coordination strength score

Make CSS to changes in identities of swinging legs<br>
For example: <br>
L1R2 → L3R2 with a seamless transition is now one bout of tetrapod_canonical rather than two

Export code to analyzeSteps.py on 29 July 2024 (this code in analyzeSteps is likely updated and more current now)

In [2]:
os.chdir('/Users/iwoods/Desktop')

In [3]:
df = pd.read_excel('testing.xlsx', sheet_name = 'gait_styles', index_col=False)

In [4]:
df.head(3)

,frametimes,speed (mm/s),speed (bodylength/s),gaits_lateral,swinging_lateral,gaits_rear,swinging_rear,tetrapod_coordination,tetrapod_speed,tripod_coordination,tripod_speed
0,0.0303,0.045143,0.171263,pentapod,L1,step,R4,NaN,NaN,NaN,NaN
1,0.0606,0.046496,0.176394,pentapod,L1,step,R4,NaN,NaN,NaN,NaN
2,0.0909,0.047800,0.181342,pentapod,L1,step,R4,NaN,NaN,NaN,NaN


In [5]:
'''
Get vector of leg combos for each frame
Get vector of gait_styles for each frame
Get vector of speeds for each frame
'''

speed_vec = df['speed (bodylength/s)'].values
gaits_vec = df['gaits_lateral'].values
combo_vec = df['swinging_lateral'].values

In [48]:
def legsInCombo(legs, combo):
    '''
    Function to see if any leg in a list of legs is in a particular combo
        inputs = 
            legs = a list of legs (e.g. ['L1','R3'])
            combo = a string (e.g. 'L1_R2')
    '''
    
    try: 
        np.isnan(combo)
        return False
    
    except:
        for leg in legs:
            if leg in combo:
                return True
        return False

def css(combo_vec, start_bout, end_bout):
    
    '''
    Function to calculate coordination score based on boundaries
        input = boundaries and vector of leg combos for each frame
        Get earliest start for combo at beginning
        Get latest end for combo at end
        Calculate coordination strength
    '''
    
    # move back from start_bout to get earliest swing of legs in swinging combo
    back_counter = 0
    
    legs_at_start = combo_vec[start_bout].split('_')
    found_start = False
    
    while found_start is False:
        
        idx = start_bout - back_counter - 1
        if idx >= 0:
            
            combo = combo_vec[idx]
            if legsInCombo(legs_at_start, combo):
                back_counter += 1
            else:
                found_start = True
                
        else:
            found_start = True
    
    # move forward from end_bout to get latest swing of legs in swinging combo
    forward_counter = 0
    
    legs_at_end = combo_vec[end_bout].split('_')
    found_end = False
    
    while found_end is False:
        
        idx = end_bout + forward_counter + 1
        if idx < len(combo_vec):
            
            combo = combo_vec[idx]
            if legsInCombo(legs_at_end, combo):
                forward_counter += 1
            else:
                found_end = True
            
        else:
            found_end = True
            
    bout_duration = end_bout + 1 - start_bout
    tot_duration = bout_duration + back_counter + forward_counter
    
    coordination_strength = bout_duration / tot_duration
    
    return coordination_strength, bout_duration

'''
Function to calculate speed based on boundaries
    input = boundaries and vector of speeds for each frame
'''

def speedInBout(speed_vec, start_bout, end_bout):
    return np.mean(speed_vec[start_bout:end_bout+1])

#print(legsInCombo(['L1','R1','L3'], 'L2_L3'))

# testing functions

In [41]:
# print(combo_vec[12:24])
# print(combo_vec[13:22])
# print(combo_vec[21])`

print(css(combo_vec, 13, 21)) # should be 1, 9 OK
print(css(combo_vec, 33, 36)) # should be 0.307, 4  OK
print(css(combo_vec, 37, 39)) # should be 0.25, 3 OK 
print(css(combo_vec, 43, 44)) # should be 0.095, 2 OK 
print(css(combo_vec, 49, 56)) # should be 0.533, 8 OK 

print()
print(speedInBout(speed_vec, 13, 21)) # should be 0.202 OK
print(speedInBout(speed_vec, 33, 36)) # should be 0.153 OK

(1.0, 9)
(0.3076923076923077, 4)
(0.25, 3)
(0.09523809523809523, 2)
(0.5333333333333333, 8)

0.2021144864836111
0.15270636842983162


In [31]:
test_vec = combo_vec
test_vec[:4] = 'L2_R1'
print(test_vec[:10])
print(css(test_vec, 0, 4)) # testing for zeros at beginning

test_vec[-5:] = 'L2_R1'
print(len(test_vec))
print(test_vec[332:]) # testing for things bigger than fector
print(css(test_vec, 332, 336))

['L2_R1' 'L2_R1' 'L2_R1' 'L2_R1' 'L1' 'L1' 'L1' 'L1' nan nan]
(0.625, 5)
337
['L2_R1' 'L2_R1' 'L2_R1' 'L2_R1' 'L2_R1']
(1.0, 5)


In [ ]:
# main code

In [63]:
# function to see if there is any intersection between two lists
def common_member(a, b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(True) 
    return(False) 

def getCssAndSpeeds(gait_styles_df, gait_style):
    
    new_df = gait_styles_df.copy()
    
    # define searchterm
    searchterm = gait_style + '_canonical'
    
    # get vectors we need
    speed_vec = gait_styles_df['speed (bodylength/s)'].values
    gaits_vec = gait_styles_df['gaits_lateral'].values
    combo_vec = gait_styles_df['swinging_lateral'].values
    
    # set up vectors for return
    coord_strength_vec = np.empty(len(gaits_vec))
    coord_strength_vec[:] = np.nan
    coord_speed_vec = np.empty(len(gaits_vec))
    coord_speed_vec[:] = np.nan
    
    # set up column names for updated vector
    coord_col_name = gait_style + '_coordination'
    speed_col_name = gait_style + '_speed'

    current_combo = ''
    for i, frame in enumerate(gaits_vec):
        if frame == searchterm: # if X_canonical

            if len(current_combo) > 0: # if have a current combo

                if frame == current_combo: # this is the same combo
                    end_bout = i # update end to this frame

                else: # this is a different combo
                    currentcombo_list = current_combo.split('_')
                    newcombo_list = combo_vec[i].split('_')

                    if common_member(currentcombo_list, newcombo_list): # If overlap with current combo
                        current_combo = combo_vec[i] # Update current combo
                        end_bout = i # update end to this frame

                    else: # No overlap with current combo, all done with old one

                        # Calculate things based on beginning and end
                        coordination_strength, bout_duration = css(combo_vec, begin_bout, end_bout)
                        avg_bout_speed = speedInBout(speed_vec, begin_bout, end_bout)

                        # Insert into CSS and Boutspeed vectors at beginning index
                        coord_strength_vec[i] = coordination_strength
                        coord_speed_vec[i] = avg_bout_speed

                        # testing
                        # print(begin_bout, end_bout, coordination_strength, avg_bout_speed)

                        # done ... set current combo to this new one
                        current_combo = frame

            else: # do not have a current combo
                current_combo = combo_vec[i] # Update current combo
                begin_bout = i # update beginning to this frame
                end_bout = i # update end to this frame

        else: # not X_canonical

            if len(current_combo) > 0: # if have a current combo, all done with it

                # Calculate things based on beginning and end
                coordination_strength, bout_duration = css(combo_vec, begin_bout, end_bout)
                avg_bout_speed = speedInBout(speed_vec, begin_bout, end_bout)

                # Insert into CSS and Boutspeed vectors at beginning index
                coord_strength_vec[i] = coordination_strength
                coord_speed_vec[i] = avg_bout_speed

                # testing
                # print(begin_bout, end_bout, coordination_strength, avg_bout_speed)

                # done ... empty out current combo
                current_combo = ''
    
    # finished going through frames, add columns of new data to dataframe
    new_df[coord_col_name] = coord_strength_vec
    new_df[speed_col_name] = coord_speed_vec
    
    return new_df

my_new_thing = getCssAndSpeeds(df, 'tetrapod')
my_new_thing = getCssAndSpeeds(my_new_thing, 'tripod')

In [64]:
my_new_thing.columns

Index(['frametimes', 'speed (mm/s)', 'speed (bodylength/s)', 'gaits_lateral',
       'swinging_lateral', 'gaits_rear', 'swinging_rear',
       'tetrapod_coordination', 'tetrapod_speed', 'tripod_coordination',
       'tripod_speed'],
      dtype='object')

In [67]:
print(my_new_thing['tetrapod_speed'].values)

[       nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan 0.20211449        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan 0.15392818        nan
        nan        nan        nan 0.16459186        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan 0.16907991        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan 0.16860979        nan        nan        nan
        nan 0.17001991        nan        nan        nan        nan
        nan        nan        nan        nan        nan        nan
        nan        nan        nan        nan        nan       